In [1]:
import json
import re
import tqdm
from IPython.display import Markdown

from bz2file import BZ2File
from wiktionary_de_parser import Parser

from parse import *

import wikitextparser as wtp
import numpy as np
import unidecode

In [2]:
def extract_data(wikitext: str, title: str):

    data = {
        'word': title
    }

    splitted_text = wikitext.split("\n")

    senses = []
    synonyms = []
    antonyms = []
    num_translations = 0

    senses_on = False
    synonyms_on = False
    antonyms_on = False
    translations_on = False
    first_translation_line = False

    for i, line in enumerate(splitted_text):

        if senses_on:
            try:
                sense = parse(":[{}] {}", line)[1]
                sense = sense.replace("]","")
                sense = sense.replace("[","")
                senses.append(sense)
                continue
            except:
                senses_on = False
        
        if synonyms_on:
            try:
                synonym = parse(":[{}] {}", line)[1]
                synonym = synonym.replace("]","")
                synonym = synonym.replace("[","")
                synonyms.append(synonym)
                continue
            except:
                synonyms_on = False
        
        if antonyms_on:
            try:
                antonym = parse(":[{}] {}", line)[1]
                antonym = antonym.replace("]","")
                antonym = antonym.replace("[","")
                antonyms.append(antonym)
                continue
            except:
                antonyms_on = False
        
        if translations_on:
            if first_translation_line:
                first_translation_line = False
                continue
            try:
                _, __, ___ = parse("{} [{}] {}", line)
                num_translations += 2 # very simple heuristic, assuming there are two translation items in each row :D
                continue
            except:
                translations_on = False

        if "{{Bedeutungen}}" in line:
            senses_on = True
            continue

        if "{{Gegenwörter}}" in line:
            antonyms_on = True
            continue

        if "{{Synonyme}}" in line:
            synonyms_on = True
            continue

        if "{{Übersetzungen}}" in line:
            translations_on = True
            first_translation_line = True
            continue
    
    data['senses'] = senses
    data['synonyms'] = synonyms
    data['antonyms'] = antonyms
    data['num_translations'] = num_translations

    
    return data


In [3]:
bzfile_path = "dewiktionary-20210201-pages-articles.xml.bz2"
bz = BZ2File(bzfile_path)

db = {}

for i, record in enumerate(tqdm.tqdm(Parser(bz))):
    if 'langCode' not in record or record['langCode'] != 'de':
        continue

    word = record['title']
    word = word.replace("ä", "ae")
    word = word.replace("ü", "ue")
    word = word.replace("ö", "oe")
    word = word.replace("ß", "ss")

    
    data = extract_data(record['wikitext'], word)
    

    title = word.lower()
    key = title

    j = 0

    while key in db:
        key = title + str(j)
        j += 1
    
    db[key] = data

    i += 1



989351it [04:32, 3636.28it/s]


## filter database from links and only keep "relevant" items

In [6]:
def clean_text(s:str):

    if s.startswith("{{"):
        raw_categories, text = parse("{{{{{}}}}}{}", s)
        categories = raw_categories.split('|')
        if "K" in categories:
            categories.remove("K")
        
        categories_text = f"({', '.join(categories)}):"
    else:
        categories_text = ""
        text = s

    cleaned_text = text
    while "<ref " in cleaned_text and not "</ref>" in cleaned_text:
        pre = ""
        post = ""
        try:
            pre, _, post = parse("{}<ref{}/>{}", cleaned_text)
        except:
            try:
                _, post = parse("<ref{}/>{}", cleaned_text)
            except:
                try:
                    pre, _ = parse("{}<ref{}/>", cleaned_text)
                except:
                    print("cannot clean inline ref from text")
                    break
        cleaned_text = pre + post

    while "<ref" in cleaned_text:
        pre = ""
        post = ""
        try:
            pre, _, post = parse("{}<ref{}</ref>{}", cleaned_text)
        except:
            try:
                _, post = parse("<ref{}</ref>{}", cleaned_text)
            except:
                try:
                    pre, _ = parse("{}<ref{}</ref>", cleaned_text)
                except:
                    print("cannot clean ref from text")
                    break
        cleaned_text = pre + post

    cleaned_text = cleaned_text.replace("'","")

    text = categories_text + cleaned_text

    text = text.replace("::", ":")

    return text



In [7]:
filtered_db = {}

for key, item in tqdm.tqdm(db.items()):
    senses = item['senses']
    synonyms = item['synonyms']
    antonyms = item['antonyms']

    def filter_entries(entries: list):
        new_list = []
        for entry in entries:
            try:
                cleaned_entry = clean_text(entry)
                # sort out bad hints:
                if "{" in cleaned_entry:
                    continue
                if item['word'].lower() in cleaned_entry.lower():
                    continue
                if "familienname" in cleaned_entry.lower():
                    continue
                if "ortsteil von" in cleaned_entry.lower():
                    continue
                if "dorf in" in cleaned_entry.lower():
                    continue
                if entry.isupper(): # try to sort out initialisms
                    continue
                
                cleaned_entry = cleaned_entry.replace("|","/")

                new_list.append(cleaned_entry)
            except:
                #print("cannot process item", entry)
                # just skipping unprocessable items
                pass

    
        return new_list
    
    item['senses'] = filter_entries(senses)
    item['synonyms'] = filter_entries(synonyms)
    item['antonyms'] = filter_entries(antonyms)

    # clean key from special characters:
    unaccented_key = unidecode.unidecode(key)
    unaccented_word = unidecode.unidecode(item['word'])

    def has_digit(s: str):
        for c in s:
            if c.isdigit():
                return True
        return False

    n_hints = len(item['senses']) + len(item['synonyms']) + len(item['antonyms'])

    if (n_hints == 1 and item['num_translations'] >= 6) or n_hints > 1:
        if not has_digit(item['word']):
            item['word'] = unaccented_word
            filtered_db[unaccented_key] = item


  1%|▏         | 9931/726049 [00:00<00:13, 51192.22it/s]cannot clean ref from text
cannot clean ref from text
cannot clean ref from text
cannot clean ref from text
cannot clean ref from text
  3%|▎         | 22420/726049 [00:00<00:12, 58324.47it/s]cannot clean ref from text
cannot clean ref from text
cannot clean ref from text
cannot clean ref from text
  5%|▍         | 35021/726049 [00:00<00:11, 60170.85it/s]cannot clean ref from text
cannot clean ref from text
cannot clean ref from text
cannot clean ref from text
cannot clean ref from text
cannot clean ref from text
cannot clean ref from text
cannot clean ref from text
cannot clean ref from text
cannot clean ref from text
cannot clean inline ref from text
cannot clean ref from text
cannot clean ref from text
cannot clean ref from text
 10%|▉         | 71566/726049 [00:00<00:05, 112907.67it/s]cannot clean ref from text
cannot clean ref from text
cannot clean ref from text
cannot clean ref from text
cannot clean ref from text
cannot cl

In [13]:
db['muenchen']

{'word': 'Muenchen',
 'senses': ['Stadt in Deutschland, die Hauptstadt von Bayern',
  'Ortsteil von Bad Berka in Thüringen',
  'Ortsteil von Hutthurm in Bayern',
  'Ortsteil von Hirschbach in Bayern',
  'Ortsteil von Uebigau-Wahrenbrück in Brandenburg',
  'Dorf in Nordböhmen (Mnichov u Lučního Chvojna)',
  'Wüstung (Mnichov) bei Zahrádky u České Lípy in Nordböhmen'],
 'synonyms': [],
 'antonyms': [],
 'num_translations': 72}

In [16]:
len(filtered_db)

64385

In [18]:
with open('de.json', 'w') as f:
    json.dump(filtered_db, f, indent = 4)

In [25]:
l = [1,2,3]
l.remove(4)
l

ValueError: list.remove(x): x not in list